In [10]:
import pandas as pd
import numpy as np



In [11]:
data = pd.read_csv('./ml-1m/ratings.dat', sep='::', header=None)
data.columns = ['session_id', 'item_id', 'rating', 'time']

data['session_id'] = data['session_id'] - 1
data['item_id'] = data['item_id'] - 1

num_items = 3952
print(num_items)
groups = data.groupby('session_id')

ids = data.session_id.unique()

data_grouped = pd.DataFrame()

for id in ids:
    group = groups.get_group(id)

    group = group.sort_values('time')

    data_grouped = pd.concat([data_grouped, group], axis=0)

data_grouped

/home/yangzy/anaconda3/envs/exp/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


3706


,session_id,item_id,rating,time
31,0,3185,4,978300019
27,0,1720,4,978300055
37,0,1021,5,978300055
22,0,1269,5,978300055
24,0,2339,3,978300103
...,...,...,...,...
1000019,6039,2916,4,997454429
1000172,6039,1783,3,997454464
999988,6039,1920,4,997454464
1000167,6039,160,3,997454486


In [12]:
l3_session = data_grouped
total_ids = l3_session.session_id.unique()

groups = l3_session.groupby('session_id')

id_time = dict()

for id in total_ids:
    group = groups.get_group(id)
    begin_time = min(group['time'])
    id_time[id] = begin_time

time_ids = sorted(id_time.items(), key = lambda kv:(kv[1], kv[0]))
# print(time_ids)

sorted_ids = [i[0] for i in time_ids]

fractions = np.array([0.8, 0.1, 0.1])

train_ids, val_ids, test_ids = np.array_split(sorted_ids, (fractions[:-1].cumsum() * len(total_ids)).astype(int))

train_sessions = l3_session[l3_session['session_id'].isin(train_ids)]
val_sessions = l3_session[l3_session['session_id'].isin(val_ids)]
test_sessions = l3_session[l3_session['session_id'].isin(test_ids)]

train_sessions.to_pickle('./train_sessions.df')
val_sessions.to_pickle('./val_sessions.df')
test_sessions.to_pickle('./test_sessions.df')

print(train_sessions)
print(val_sessions)
print(test_sessions)

         session_id  item_id  rating       time
194342         1198      919       4  974846442
194424         1198     1720       3  974846459
194427         1198     2532       4  974846477
194334         1198     1269       5  974846477
194464         1198      355       4  974846499
...             ...      ...     ...        ...
1000019        6039     2916       4  997454429
1000172        6039     1783       3  997454464
999988         6039     1920       4  997454464
1000167        6039      160       3  997454486
1000042        6039     1220       4  998315055

[804102 rows x 4 columns]
        session_id  item_id  rating        time
90466          598     3734       3   975884811
90457          598     3872       3   975884811
90449          598      907       4   975884811
90463          598      355       3   975884854
90452          598     3419       4   975884854
...            ...      ...     ...         ...
384747        2246     2392       4  1034474994
384770       

In [13]:
from numpy.lib.arraypad import pad

def pad_history(itemlist,length,pad_item):
    if len(itemlist)>=length:
        return itemlist[-length:]
    if len(itemlist)<length:
        temp = [pad_item] * (length-len(itemlist))
        itemlist.extend(temp)
        return itemlist


length = 10

# valid_l3_session = data
# valid_items = list(valid_l3_session['item_id'].unique())
# # item_ids = valid_items.iloc[:, 0]
pad_item = num_items

train_sessions = pd.read_pickle('./train_sessions.df')
groups = train_sessions.groupby('session_id')

ids = train_sessions.session_id.unique()

state, len_state, action, is_read, next_state, len_next_state, is_done = [], [], [], [], [],[],[]

for id in ids:
    group = groups.get_group(id)
    history = []
    for index, row in group.iterrows():
        s = list(history)
        len_state.append(length if len(s) >= length else 1 if len(s) == 0 else len(s))
        s = pad_history(s, length, pad_item)
        a = row['item_id']
        is_r = row['time']
        state.append(s)
        action.append(a)
        is_read.append(is_r)
        history.append(row['item_id'])
        # next_s = list(history)
        # len_next_state.append(length if len(next_s)>=length else 1 if len(next_s)==0 else len(next_s))
        # next_s = pad_history(next_s, length, pad_item)
        # next_state.append(next_s)
        is_done.append(False)
    is_done[-1] = True

dic = {'seq':state,'len_seq':len_state,'next':action}

replay_buffer=pd.DataFrame(data=dic)
replay_buffer.to_pickle('./train_data.df')

dic = {'seq_size':[length],'item_num':[pad_item]}
data_statis = pd.DataFrame(data=dic)
data_statis.to_pickle('./data_statis.df')


In [14]:
def pad_history(itemlist,length,pad_item):
    if len(itemlist)>=length:
        return itemlist[-length:]
    if len(itemlist)<length:
        temp = [pad_item] * (length-len(itemlist))
        itemlist.extend(temp)
        return itemlist
    
# valid_l3_session = data
# valid_items = list(valid_l3_session['item_id'].unique())
# # item_ids = valid_items.iloc[:, 0]
pad_item = num_items
length = 10

state, len_state, action = [], [], []

ids = val_sessions.session_id.unique()

groups = val_sessions.groupby('session_id')

for id in ids:
    group = groups.get_group(id)
    history = []
    for index, row in group.iterrows():
        s = list(history)
        len_state.append(length if len(s) >= length else 1 if len(s) == 0 else len(s))
        s = pad_history(s, length, pad_item)
        a = row['item_id']
        is_r = row['time']
        state.append(s)
        action.append(a)
        history.append(row['item_id'])
        # next_s = list(history)
        # len_next_state.append(length if len(next_s)>=length else 1 if len(next_s)==0 else len(next_s))
        # next_s = pad_history(next_s, length, pad_item)
        # next_state.append(next_s)

val_dic = {'seq':state,'len_seq':len_state,'next':action}

val_buffer=pd.DataFrame(data=val_dic)
val_buffer.to_pickle('./val_data.df')
val_buffer

,seq,len_seq,next
0,"[3706, 3706, 3706, 3706, 3706, 3706, 3706, 370...",1,3734
1,"[3734, 3706, 3706, 3706, 3706, 3706, 3706, 370...",1,3872
2,"[3734, 3872, 3706, 3706, 3706, 3706, 3706, 370...",2,907
3,"[3734, 3872, 907, 3706, 3706, 3706, 3706, 3706...",3,355
4,"[3734, 3872, 907, 355, 3706, 3706, 3706, 3706,...",4,3419
...,...,...,...
104657,"[1253, 3329, 3094, 3151, 918, 904, 1303, 953, ...",10,2392
104658,"[3329, 3094, 3151, 918, 904, 1303, 953, 915, 3...",10,198
104659,"[3094, 3151, 918, 904, 1303, 953, 915, 3102, 2...",10,1013
104660,"[3151, 918, 904, 1303, 953, 915, 3102, 2392, 1...",10,3858


In [15]:
def pad_history(itemlist,length,pad_item):
    if len(itemlist)>=length:
        return itemlist[-length:]
    if len(itemlist)<length:
        temp = [pad_item] * (length-len(itemlist))
        itemlist.extend(temp)
        return itemlist
    
# data_statis = pd.read_pickle('./l2l3/data_statis20.df')  # read data statistics, includeing seq_size and item_num
# length = 20  # the length of history to define the seq
# pad_item = data_statis['item_num'][0]  # total number of items

test_sessions = pd.concat([val_sessions, test_sessions], axis=0)

state, len_state, action = [], [], []

ids = test_sessions.session_id.unique()

groups = test_sessions.groupby('session_id')

for id in ids:
    group = groups.get_group(id)
    history = []
    for index, row in group.iterrows():
        s = list(history)
        len_state.append(length if len(s) >= length else 1 if len(s) == 0 else len(s))
        s = pad_history(s, length, pad_item)
        a = row['item_id']
        is_r = row['time']
        state.append(s)
        action.append(a)
        history.append(row['item_id'])
        # next_s = list(history)
        # len_next_state.append(length if len(next_s)>=length else 1 if len(next_s)==0 else len(next_s))
        # next_s = pad_history(next_s, length, pad_item)
        # next_state.append(next_s)

test_dic = {'seq':state,'len_seq':len_state,'next':action}

test_buffer=pd.DataFrame(data=test_dic)
test_buffer.to_pickle('./ttest_data.df')
test_buffer

,seq,len_seq,next
0,"[3706, 3706, 3706, 3706, 3706, 3706, 3706, 370...",1,3734
1,"[3734, 3706, 3706, 3706, 3706, 3706, 3706, 370...",1,3872
2,"[3734, 3872, 3706, 3706, 3706, 3706, 3706, 370...",2,907
3,"[3734, 3872, 907, 3706, 3706, 3706, 3706, 3706...",3,355
4,"[3734, 3872, 907, 355, 3706, 3706, 3706, 3706,...",4,3419
...,...,...,...
196102,"[2616, 857, 1220, 3467, 1261, 259, 2761, 1926,...",10,2760
196103,"[857, 1220, 3467, 1261, 259, 2761, 1926, 1243,...",10,3184
196104,"[1220, 3467, 1261, 259, 2761, 1926, 1243, 3915...",10,3128
196105,"[3467, 1261, 259, 2761, 1926, 1243, 3915, 2760...",10,3482


In [16]:
def pad_history(itemlist,length,pad_item):
    if len(itemlist)>=length:
        return itemlist[-length:]
    if len(itemlist)<length:
        temp = [pad_item] * (length-len(itemlist))
        itemlist.extend(temp)
        return itemlist
    
# data_statis = pd.read_pickle('./l2l3/data_statis20.df')  # read data statistics, includeing seq_size and item_num
# length = 20  # the length of history to define the seq
# pad_item = data_statis['item_num'][0]  # total number of items

state, len_state, action = [], [], []


ids = test_sessions.session_id.unique()

groups = test_sessions.groupby('session_id')

for id in ids:
    group = groups.get_group(id)
    history = []
    for index, row in group.iterrows():
        # s = list(history)
        # len_state.append(length if len(s) >= length else 1 if len(s) == 0 else len(s))
        # s = pad_history(s, length, pad_item)
        # a = row['item_id']
        # is_r = row['time']
        # state.append(s)
        # action.append(a)
        history.append(row['item_id'])
        # next_s = list(history)
        # len_next_state.append(length if len(next_s)>=length else 1 if len(next_s)==0 else len(next_s))
        # next_s = pad_history(next_s, length, pad_item)
        # next_state.append(next_s)
    state.append(pad_history(history[:-1], length, pad_item))
    len_state.append(length if len(history[:-1]) >= length else 1 if len(history[:-1]) == 0 else len(history[:-1]))
    action.append(history[-1])

test_dic = {'seq':state,'len_seq':len_state,'next':action}

test_buffer=pd.DataFrame(data=test_dic)
test_buffer.to_pickle('./Test_data.df')
test_buffer

,seq,len_seq,next
0,"[317, 2924, 1949, 526, 1220, 1275, 1227, 1192,...",10,1246
1,"[3174, 1958, 2960, 2700, 2986, 967, 3175, 2996...",10,3645
2,"[17, 2278, 2966, 2604, 2125, 3062, 2121, 2902,...",10,2105
3,"[2721, 2274, 1689, 184, 255, 2671, 3695, 3877,...",10,2985
4,"[376, 1832, 1607, 2762, 647, 1046, 779, 2272, ...",10,1478
...,...,...,...
1203,"[3743, 3862, 3299, 3554, 3561, 3512, 3285, 386...",10,3645
1204,"[3453, 3615, 3753, 3315, 3298, 3323, 3578, 374...",10,3914
1205,"[1135, 1079, 2787, 1088, 295, 1728, 1254, 2166...",10,3665
1206,"[1718, 507, 2290, 456, 1570, 1776, 355, 2317, ...",10,1731


In [17]:
print(num_items, max(list(data['item_id'])))

3706 3951
